In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import time 
# from __future__ import print_function, division
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

import lightgbm as lgb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
start=time.time()
train_t=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\analytics_vidhya\Customer segmentation\train.csv',chunksize=1000)
test_t=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\analytics_vidhya\Customer segmentation\test.csv',chunksize=1000)
sample_t=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\Hackerearth\analytics_vidhya\Customer segmentation\sample.csv',chunksize=1000)
end=time.time()
train=pd.concat(train_t)
test=pd.concat(test_t)
sample=pd.concat(sample_t)
print(end-start,'sec')

0.02101302146911621 sec


In [13]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [14]:

train = train.fillna('')
#df = train.copy(deep=True)
df =train
test = test.fillna('')

le = LabelEncoder()
categorical = np.array(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Var_1', 'Spending_Score', 'Segmentation'])

for i in range(len(categorical)):
    train[categorical[i]] = le.fit_transform(train[categorical[i]])
    if i < len(categorical)-1:
        test[categorical[i]] = le.fit_transform(test[categorical[i]])



In [15]:
train = train.replace('', np.nan)
test = test.replace('', np.nan)

df_test = test.copy(deep=True)

labels = train['Segmentation'].values
train = train.drop(['Segmentation'], axis = 1)
train = train.drop(['ID'], axis = 1)
test = test.drop(['ID'], axis = 1)

temp, age_temp = list(df['Age'].values), list()
for age in range(len(temp)):
    i = temp[age]
    if i <=33:
        age_temp.append(0)
    elif i <=67:
        age_temp.append(1)
    elif i <=89:
        age_temp.append(2)

train['Age_Prof'] = age_temp

temp, age_temp = list(df_test['Age'].values), list()
for age in range(len(temp)):
    i = temp[age]
    if i <=33:
        age_temp.append(0)
    elif i <=67:
        age_temp.append(1)
    elif i <=89:
        age_temp.append(2)

test['Age_Prof'] = age_temp

train = train.values
test = test.values

scaler = StandardScaler()
X_train = train
y_train = labels
scaler.fit(X_train)

X_train = scaler.transform(X_train)
models_rf, models_lgb = list(), list()
kf = KFold(n_splits=5, random_state=28, shuffle=False)
for train_index, test_index in kf.split(X_train):
    X_traink, X_testk = X_train[train_index], X_train[test_index]
    y_traink, y_testk = y_train[train_index], y_train[test_index]
    clf2 = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=80, min_data_in_leaf=307, max_iter=1000, max_depth=7, learning_rate=0.1)
    clf2.fit(X_traink, y_traink)
    models_lgb.append(clf2)
    scores = clf2.predict(X_testk)
    #print(accuracy_score(scores, y_testk))

best_model_lgb = models_lgb[1]

test = scaler.transform(test)
preds2 = best_model_lgb.predict(test)

final_preds2, temp_train, temp_test = list(), list(df["ID"].values), list(df_test["ID"].values)
for i in range(len(temp_test)):
    id_ = temp_test[i]
    if (id_ in temp_train):
        temp = df[df['ID'] == id_]
        final_preds2.append(temp['Segmentation'].values[0])
    else:
        final_preds2.append(preds2[i])

new_preds2 = []

for i in range(len(preds2)):
    if final_preds2[i] == 0:
        new_preds2.append('A')
    elif final_preds2[i] == 1:
        new_preds2.append('B')
    elif final_preds2[i] == 2:
        new_preds2.append('C')
    elif final_preds2[i] == 3:
        new_preds2.append('D')
        

df_submit = pd.DataFrame({'ID': df_test['ID'].values, 'Segmentation':new_preds2})
df_submit.to_csv('Customer_seg_submit2.csv', index = False)

StandardScaler()

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


LGBMClassifier(max_depth=7, max_iter=1000, min_data_in_leaf=307, num_leaves=80)

LGBMClassifier(max_depth=7, max_iter=1000, min_data_in_leaf=307, num_leaves=80)

LGBMClassifier(max_depth=7, max_iter=1000, min_data_in_leaf=307, num_leaves=80)

LGBMClassifier(max_depth=7, max_iter=1000, min_data_in_leaf=307, num_leaves=80)

LGBMClassifier(max_depth=7, max_iter=1000, min_data_in_leaf=307, num_leaves=80)